In [141]:
import numpy as np
from hmmlearn.hmm import MultinomialHMM

#Under cts IRT model; not for BKT use
class Question:
  def __init__(self, difficulty, concept):
    self.d = difficulty
    self.c = concept

  def pRight(self, skill):
    return 1.0/(1.0 + np.exp(self.d - skill))


def ber(p):
  return np.random.binomial(1,p)

# This is... 
def pRight(diff, skill):
  return .25 + .75/(1.0 + np.exp(diff - skill))


pL0 = 0.3 # Probability of knowing the concept initially
pG = 0.25 # Probability of guessing correctly 
pS = .05 # Probability of slipping (answering incorrectly despite knowing the concep)
pT = 0.1 # Probability of learning the concept after an incorrect answer

numStudents = 100
numQuestions = 300
numConcepts = 2

conceptDiffs = np.random.normal(0,1,numConcepts)
qConcepts = np.random.randint(0,numConcepts, numQuestions)
qMeans = np.array([conceptDiffs[qConcept] for qConcept in qConcepts])
diffs = np.random.normal(0,1, numQuestions) + qMeans
answers = np.zeros((numStudents, numQuestions))
knowTracks = np.zeros((numStudents, numConcepts, numQuestions))

# Not sure what this is doing exactly
for i in range(numStudents):
    know = np.random.binomial(1, pL0, numConcepts)
    for j in range(numQuestions):
        curConcept = qConcepts[j]
        #if i == 0 and curConcept == 0: print know[0]
        knowTracks[i, curConcept, j] = know[curConcept]
        knowTracks[i, :, j] = know
        answers[i,j] = ber(pRight(diffs[j], know[curConcept]))
        if ber(pT) == 1: know[curConcept] = 1

#print "concept difficulties", conceptDiffs
#print "question concepts", qConcepts
#print "question diffs", diffs
#print answers

def multinomify(traj):
    # This way of formatting the data is taken from https://github.com/hmmlearn/hmmlearn/issues/70
    return np.array([[int(t) for t in traj]]).T

#Now, infer HMM parameters from emissions:

model = MultinomialHMM(n_components=2)
student = 0
concept = 0
trainData = multinomify(answers[student, qConcepts == concept])
M = model.fit(trainData)
print "Emission probabilities are:"
print M.emissionprob_

print "The slip probability is: %.02f" % M.emissionprob_[1][0]
print "The probability of a correct guess is: %.02f" % M.emissionprob_[0][1]

#print "Transmission probabilities are:"
#print M.transmat_
M.predict(trainData)

Emission probabilities are:
[[ 0.7085658   0.2914342 ]
 [ 0.01919083  0.98080917]]
The slip probability is: 0.02
The probability of a correct guess is: 0.29


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1])

In [124]:
#trainData = []
#trainLengths = []
#for s in range(numStudents):
#  for c in range(numConcepts):
#    trajectories = answers[s,qConcepts == c]
#    trainData += multinomify(trajectories)
#    trainLengths.append(len(trajectories))
    
#M = model.fit(trainData, trainLengths)